In [1]:
import pandas as pd
import ast
from pprint import pprint
import json

In [2]:
a,count = 0,0
failed_parse = []


def create_out_string(inp):
    s = "{"
    # print(inp)
    for e in inp:
        # print(e)
        for k, v in e.items():
            s += f"{k}: {v}; "

        s = s[:-2] + " | "

    return s[:-3] + "}"

def format_labels(label: str):
    nlabel =  label.lower().replace('"', '').replace(';',',').replace('decision: ', "\'decision\': '").replace('subject: ', "\'subject\': '").replace('action: ', "\'action\': '").replace('resource: ', "\'resource\': '").replace('condition: ', "\'condition\': '").replace('purpose: ', "\'purpose\': '").replace(",", "',").replace('}', "'}").replace("'s ", "\\'s ")
    i = nlabel.find('decision')
    return ("{'" + nlabel[i:])

def postprocess(key, val):
    
    l = ['any', 'all', 'every','the']
    conds = ['when', 'if', 'if,']
    purs = ['for', 'to']
    
    val = val.replace('_',',').replace('-', '').replace("'", "").replace('’', '').replace("”", "").replace("“","")
    
    if key == 'subject' or key == 'resource':
        for k in l:
            if val.split(' ')[0] == k:
                val = ' '.join(val.split(' ')[1:])
            
    if key=='purpose' and val.split(' ')[0] in purs:
        val = ' '.join(val.split(' ')[1:])
        
    if key=='condition' and val.split(' ')[0] in conds:
        val = ' '.join(val.split(' ')[1:])
    
        
    return ' '.join(val.split())


def make_json(labels):
    global count, a, failed_parse
    policies = []
    formatted_list = []
    srls = {}
    for l in labels:
        formatted = format_labels(l)
        formatted_list.append(formatted)
    
    for f in list(set(formatted_list)):
        a+=1
        try:
            label_json = ast.literal_eval(f)
            pp = {'decision': 'allow', 'subject': 'none', 'action': 'none', 'resource': 'none', 'condition': 'none', 'purpose': 'none'}
            for key,val in label_json.items():
                key = key.strip()
                if key in pp:
                    pp[key] = postprocess(key, val.strip())
            policies.append(pp)
            
            action = pp['action']
            if action not in srls:
                srls[action] = {'subject': [], 'resource': [], 'purpose': [], 'condition': []}
                if pp['subject']!='none':
                    srls[action]['subject'].append(pp['subject'])
                if pp['resource']!='none':
                    srls[action]['resource'].append(pp['resource'])
                if pp['purpose']!='none':
                    srls[action]['purpose'].append(pp['purpose'])
                if pp['condition']!='none':
                    srls[action]['condition'].append(pp['condition'])    
            else:
                if pp['subject']!='none':
                    srls[action]['subject'].append(pp['subject'])
                if pp['resource']!='none':
                    srls[action]['resource'].append(pp['resource'])
                if pp['purpose']!='none':
                    srls[action]['purpose'].append(pp['purpose'])
                if pp['condition']!='none':
                    srls[action]['condition'].append(pp['condition'])
        
        except:
            count+=1
            failed_parse.append([labels, f])
            continue
        
    p = []
    for pol in policies:
        if pol not in p:
            p.append(pol)
        
    return p, srls
            
            
def process_label(result):
    res = []
    if (len(result) > 0):
        for p in result:
            ind = p.split(" | ")
            if (len(ind) == 1):
                res.append(ind[0])
            else:
                for i in range(len(ind)):
                    if (i==0 and ind[i][-1]!="}"):
                        res.append(ind[i]+"}")
                    elif (i == len(ind)-1 and ind[i][0]!="{"):
                        res.append("{" + ind[i])
                    else:
                        res.append("{" + ind[i] + "}")
    nres = list(set(res))
    return(make_json(nres))

## Loading datasets to analyze

In [4]:
overall_test = pd.read_csv('overall/test.csv')
overall_test_acps = pd.read_csv('overall/test_acp.csv')

test_collected = pd.read_csv('document_folds/collected.csv')
test_collected_acps = pd.read_csv('document_folds/collected_acp.csv')
test_ibm = pd.read_csv('document_folds/ibm.csv')
test_ibm_acps = pd.read_csv('document_folds/ibm_acp.csv')
test_cyber = pd.read_csv('document_folds/cyber.csv')
test_cyber_acps = pd.read_csv('document_folds/cyber_acp.csv')
test_acre = pd.read_csv('document_folds/acre.csv')
test_acre_acps = pd.read_csv('document_folds/acre_acp.csv')
test_t2p = pd.read_csv('document_folds/t2p.csv')
test_t2p_acps = pd.read_csv('document_folds/t2p_acp.csv')

datasets_acps = {'collected': test_collected_acps,
                 'ibm': test_ibm_acps,
                 'cyber': test_cyber_acps,
                 'itrust_t2p':test_t2p_acps,
                 'itrust_acre': test_acre_acps,
                'overall': overall_test_acps}



In [5]:
# info = {'acrs': 0, 'allow': 0, 'deny': 0, 'subject': 0, 'action': 0, 'resource': 0, 'purpose': 0, 'condition': 0}
res = {k: {'acrs': 0, 'allow': 0, 'deny': 0, 'subject': 0, 'action': 0, 'resource': 0, 'purpose': 0, 'condition': 0} for k in datasets_acps}

for name,dataset in datasets_acps.items():
    for i,o in zip(dataset['input'], dataset['output']):
        p,_ = process_label([o])
        # print(p)
        res[name]['acrs']+=len(p)
        for r in p:
            for f,c in r.items(): 
                if f == 'decision':
                    res[name][c]+=1
                elif c!='none':
                    res[name][f]+=1
    
print(json.dumps(res, indent=3))

{
   "collected": {
      "acrs": 211,
      "allow": 168,
      "deny": 43,
      "subject": 207,
      "action": 211,
      "resource": 209,
      "purpose": 41,
      "condition": 27
   },
   "ibm": {
      "acrs": 181,
      "allow": 176,
      "deny": 5,
      "subject": 123,
      "action": 181,
      "resource": 181,
      "purpose": 11,
      "condition": 64
   },
   "cyber": {
      "acrs": 168,
      "allow": 166,
      "deny": 2,
      "subject": 115,
      "action": 168,
      "resource": 167,
      "purpose": 29,
      "condition": 27
   },
   "itrust_t2p": {
      "acrs": 631,
      "allow": 616,
      "deny": 15,
      "subject": 546,
      "action": 631,
      "resource": 630,
      "purpose": 50,
      "condition": 14
   },
   "itrust_acre": {
      "acrs": 932,
      "allow": 899,
      "deny": 33,
      "subject": 770,
      "action": 932,
      "resource": 931,
      "purpose": 79,
      "condition": 39
   },
   "overall": {
      "acrs": 571,
      "allow": 468,
  